In [ ]:
class TrainModel():
    def Model(self,channel,size):
        from keras.layers.convolutional import Convolution2D, MaxPooling2D
        from keras.models import Sequential
        from keras.layers.core import Dense, Dropout, Activation, Flatten
        model = Sequential()
        model.add(Convolution2D(32, (3, 3),
                                activation='relu',
                                input_shape=(channel, size, size),
                                data_format='channels_first'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
        model.add(Convolution2D(64, kernel_size=(3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation('relu'))
        model.add(Dense(10))
        model.add(Dropout(0.2))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=["binary_accuracy"])
        return model
    def GetDatas(self,listimg,listlabel,num_class):
        import numpy as np
        from keras.utils import np_utils
        from sklearn.utils import shuffle
        img_data = np.array(listimg)
        img_data = img_data.astype('float32')
        img_data /= 255
        labels = np.array(listlabel)
        Y = np_utils.to_categorical(labels, num_class)
        x, y = shuffle(img_data, Y, random_state=30)
        return x,y

    def MyModel(self,imgdata, labeldata, size=28, channel=1, batchsize=200, 
                epochs=15, split=0.2, num_class=10):
        import numpy as np
        from sklearn.model_selection import train_test_split
        x,y=self.GetDatas(imgdata,labeldata,num_class)
        X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=split,random_state=6)
        model=self.Model(channel,size)
        X_train = X_train.reshape(-1, channel, size, size)
        X_test = X_test.reshape(-1, channel, size, size)
        hist = model.fit(X_train,y_train,
                         batch_size=batchsize,
                         epochs=epochs,
                         verbose=1,
                         validation_data=(X_test, y_test))
        return model, hist

In [ ]:
def getDatas(size,channel=1,filter=False):
  import os
  import cv2
  import numpy as np
  from tqdm import tqdm
  KATEGORILER = ["bottle","can","chain","drink-carton","hook","propeller","shampoo-bottle","standing-bottle","tire","valve"]
  DIR = "marine-debris-watertank-release/class-image-crops"
  imgdata = []
  labeldata=[]
  for kategori in KATEGORILER:
    klasor_adresi = os.path.join(DIR,kategori)
    label = KATEGORILER.index(kategori)
    for resim_adi in tqdm(os.listdir(klasor_adresi)):
      resim_adresi = os.path.join(klasor_adresi,resim_adi)
      resim = cv2.imread(resim_adresi)
      if(resim is None):
       print("Hata")
      else:
       resim = cv2.resize(resim,(size,size))
       if(channel==1):
         resim = cv2.cvtColor(resim, cv2.COLOR_BGR2GRAY)
       if(filter):
         term = np.zeros((120, 120), dtype=np.uint8)
         term = cv2.normalize(resim, term, 0, 255, cv2.NORM_MINMAX)
         term = np.uint8(term)
         resim = cv2.applyColorMap(term, cv2.COLORMAP_JET)
       imgdata.append(resim)
       labeldata.append(label)
  return imgdata,labeldata

In [ ]:
def modelsave(smodel,path):
  from keras.models import save_model
  smodel.save(path)
def modelhistsave(hist,path):
  import pandas as pd
  if (".csv" in path):pass
  else:path+=".csv"
  df=pd.DataFrame(hist).to_csv(path)
def loadmodelhist(path):
  import pandas as pd
  return pd.read_csv(path)
def loadmodel(path):
  from keras.models import load_model
  return load_model(path)

In [ ]:
zip_adres = "/content/drive/MyDrive/Data/dataimg.zip"
!cp "{zip_adres}" .
!unzip -q dataimg.zip
!rm dataimg.zip

In [ ]:
zip_adres = "/content/drive/MyDrive/Data/YeniData.zip"
!cp "{zip_adres}" .
!unzip -q YeniData.zip
!rm YeniData.zip

In [ ]:
!zip -r YeniData1.zip YeniDatalar1

In [ ]:
!cp YeniData1.zip "/content/drive/MyDrive/Data/"

In [ ]:
def savetext(path,size,data):
    a = ["bottle", "can", "chain", "drink-carton", "hook", "propeller", "shampoo-bottle", "standing-bottle", "tire",
         "valve"]
    datasizes = [449, 367, 226, 349, 133, 137, 99, 65, 331, 208]
    top = 0
    top2 = 0
    text = ""
    listeleme=[]
    import pandas as pd
    df_train = pd.DataFrame()
    for index, item in enumerate(datasizes):
        txt = f"{a[index]} yüzde %{(data[index][index] / item) * 100}"
        df_train[a[index]]=[(data[index][index] / item) * 100];
        print(txt)
        top += data[index][index]
        top2 += item
        text += txt + "\n"
    text += f"Genel Yüzde %{(top / top2) * 100}"
    df_train["Genel Yüzde"]=[(top / top2) * 100]
    df_train["Genel Yüzde"]= [100]
    df_train.to_csv(f"{path}yüzdelikler{size}X{size}.csv")
    print(f"Genel Yüzde %{(top / top2) * 100}")
    with open(path+f'output{size}X{size}.txt', 'w') as f:
        f.write(text)
        f.close()

In [ ]:
def Test3(test_image,label,tmodel,channel=1,size=28):
  import cv2
  from matplotlib import pyplot
  import numpy as np
  sinif=np.zeros((11, 11)).astype(int)
  #test_image=cv2.resize(test_image,(size,size))
  test_image = np.array(test_image)
  test_image = test_image.astype('float32')
  test_image /= 255
  test_image = test_image.reshape(-1, channel ,size ,size)
  f=(tmodel.predict(test_image) > 0.5).astype("int32").tolist()
  liste=[]
  for ff in f:
    deger=0
    for ind,item in enumerate(ff):
      deger+=(ind+1)*item
    liste.append(deger)
  for ind,list in enumerate(liste):
    if(list==0):
      sinif[label[ind],10]+=1
      sinif[10,label[ind]]+=1
    else:
       sinif[label[ind],list-1]+=1
  return sinif;

In [ ]:
def mu(size,channel,batchsize,filtre=False):
    images, labels = getDatas(size, channel,filter=filtre)
    trainmodel = TrainModel()
    model, hist = trainmodel.MyModel(images, labels, size, channel, batchsize)
    ind = Test3(images, labels, model, channel, size)
    f="3Channel"
    if filtre:
      f="3FChannel"
    path = f"YeniDatalar1/{f}/{batchsize}BatchSize/Size{size}X{size}/"
    import pandas as pd
    df_train = pd.DataFrame()
    a = ["bottle", "can", "chain", "drink-carton", "hook", "propeller", "shampoo-bottle", "standing-bottle", "tire",
         "valve", "None"]
    for i in range(11):
        df_train[a[i]] = ind[i]
    modelsave(model, f"{path}{1}channel{size}X{size}.h5")
    df_train.to_csv(f"{path}output{size}X{size}.csv")
    df1=pd.DataFrame(hist.history).to_csv(f"{path}hist{size}X{size}.csv")
    savetext(path, size,ind)

#channel=3
#filtre=True
#sizes=[16,28,32,64,92]
#batchsizes=[15,25,50,75,100,150,200,300,400,500]

#for batchsize in batchsizes:
#  for size in sizes:
#    mu(size,channel,batchsize)
#for size in sizes:
   # mu(size,1,15)